<a href="https://colab.research.google.com/github/Ori-Cohenn/Pierian-Data-Complete-Python-3-Bootcamp/blob/master/08-Milestone%20Project%20-%202/BlackJackGame.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Milestone Project 2 - Blackjack Game
In this milestone project you will be creating a Complete BlackJack Card Game in Python.

Here are the requirements:

* You need to create a simple text-based BlackJack game
* The game needs to have one player versus an automated dealer.
* The player can stand or hit.
* The player must be able to pick their betting amount.
* You need to keep track of the player's total money.
* You need to alert the player of wins, losses, or busts, etc...

And most importantly:

* You must use OOP and classes in some portion of your game. You can not just use functions in your game. Use classes to help you define the Deck and the Player's hand. There are many right ways to do this, so explore it well!
Feel free to expand this game. Try including multiple players. Try adding in Double-Down and card splits! Remember to you are free to use any resources you want and as always:

# HAVE FUN!

In [106]:
import random # libraries and global veriables
from IPython.display import clear_output
import time
suits = ('Hearts', 'Diamonds', 'Spades', 'Clubs')
ranks = ('Two', 'Three', 'Four', 'Five', 'Six', 'Seven', 'Eight', 'Nine', 'Ten', 'Jack', 'Queen', 'King', 'Ace')
values = {'Two':2, 'Three':3, 'Four':4, 'Five':5, 'Six':6, 'Seven':7, 'Eight':8, 'Nine':9, 'Ten':10, 'Jack':10,
         'Queen':10, 'King':10, 'Ace':11} # Ace equals 11/1


In [107]:
class Card:

  def __init__(self, suit, rank):
    try:
      rank,suit = rank.capitalize(),suit.capitalize()
      self.suit = suit
      self.rank = rank
      self.value= values[rank] # value by key rank from values dic
    except:
      print("Card init failed wrong input")

  def __str__(self):
      return self.rank + ' of ' + self.suit

In [108]:
myCard = Card('hearts','two')

In [109]:
print(myCard)

Two of Hearts


In [110]:
class Deck:

  def __init__(self):
      self.all_cards=[]
      for suit in suits:
        for rank in ranks:
          self.all_cards.append(Card(suit,rank))

  def shuffle(self):
    random.shuffle(self.all_cards)

  def deal_one_card(self):
    if (len(self.all_cards)<=0):
      print("**No more cards to deal\nDeck is Empty")
      return 0
    else:
      return self.all_cards.pop(0) #deal top deck card

  def __str__(self):
        if (len(self.all_cards)<=0):
          return f'Deck is Empty'
        else:
          for i,card in enumerate(self.all_cards,1):
            print(f"#{i} {card}")
          return f'Deck has {len(self.all_cards)} cards'

In [111]:
class Player:

  def __init__(self, name:str='', balance:int=0):
    self.hand = []
    self.name = name
    self.balance = balance

  def add_card(self, new_card : Card): #HIT
    self.hand.append(new_card)

  def hand_value(self)-> int:
    value = sum(card.value for card in self.hand)
    aces = sum(1 for card in self.hand if card.rank == 'Ace') #counting Aces
    while value > 21 and aces: #setting Aces as 1 or 11
        value -= 10
        aces -= 1
    return value

  def add_bet(self)->int:
    try:
      bet = int(input(f'{self.name} please enter your bet: '))
      if (bet > self.balance):
        print(f"Not enough money to bet, Your current balance is {self.balance}$.")
        return 0
      self.balance -= bet
      return bet
    except ValueError:
      print("wrong input try again")
      return self.add_bet()

  def __str__(self)->str:
        if (len(self.hand)<=0):
          return f'{self.name} does not have any cards'
        else:
          print(f'{self.name}\'s hand:')
          for i,card in enumerate(self.hand,1):
            print(f"#{i} {card}")
          return f'Has {self.balance}$'

In [112]:
class Dealer(Player):
    def __init__(self):
        super().__init__(name='Dealer')

    def play_turn(self, deck):
        print(f"{self.name} starts with: {self.hand_value()}")
        while self.hand_value() < 17:
            self.add_card(deck.deal_one_card())
            print(f"{self.name} hits: {self.hand[-1]}")
        print(f"{self.name} stays at: {self.hand_value()}")

In [113]:
def hit_or_stand(deck : Deck, player : Player):
  try:
    choice = input("Would you like to Hit or Stand? Enter 'h' or 's': ").strip().lower()
    if choice == 'h':
        new_card = deck.deal_one_card()
        player.add_card(new_card)
        print(f"You drew: {new_card}")
        print(f"Your current hand value: {player.hand_value()}")
    elif choice == 's':
        print("You chose to stand.")
        return choice
    else:
      print("Invalid input. Please enter 'h' to hit or 's' to stand.")
      return hit_or_stand(deck, player)  # Retry for invalid input
  except Exception as e:
      print(f"An error occurred: {e}. Please try again.")
      return hit_or_stand(deck, player)  # Retry on error

In [114]:
def handle_player_turn(player: Player, deck: Deck)->bool:
    """
    Handles the player's turn (hit or stand).
    Returns True if the player busts, otherwise False.
    """
    while True:
        choice=hit_or_stand(deck, player)
        if player.hand_value() > 21:  # Player busts
            print("You bust! Dealer wins.")
            return True
        elif choice=='s' and player.hand_value() <= 21:
            print(f"Player stands at {player.hand_value()}")
            break
    return False

In [115]:
def deal_initial_cards(player: Player, dealer: Dealer, deck: Deck):
    """
    Deals two cards each to the player and the dealer.
    """
    for _ in range(2):
        player.add_card(deck.deal_one_card())
        dealer.add_card(deck.deal_one_card())


In [116]:
def display_initial_hands(player: Player,  dealer: Dealer):
    """
    Displays the player's hand and the dealer's first card.
    """
    print("\nYour hand:")
    print(player)
    print(f"Dealer shows: {dealer.hand[0]}")

In [117]:
def check_blackjack(player: Player, bet: int)->bool:
    """
    Checks if the player has a Blackjack.
    Returns True if Blackjack, otherwise False.
    """
    if player.hand_value() == 21:
        print("Blackjack! You win!")
        player.balance += int(bet * 2.5)  # Blackjack pays 3:2
        return True
    return False

In [118]:

def handle_dealer_turn(dealer: Dealer, deck: Deck):
    """
    Handles the dealer's turn (auto-play until 17 or bust).
    """
    print("\nDealer's turn:")
    print(f"Dealer's second card is: {dealer.hand[1]}")
    dealer.play_turn(deck)


In [119]:
def determine_winner(player: Player, dealer: Dealer, bet: int):
    """
    Determines the winner of the round and updates the player's balance.
    """
    if dealer.hand_value() > 21 or player.hand_value() > dealer.hand_value():
        print("You win!")
        player.balance += bet * 2
    elif player.hand_value() == dealer.hand_value():
        print("It's a tie! Bet returned.")
        player.balance += bet
    else:
        print("Dealer wins!")

In [120]:
def ask_play()-> bool:
    """
    Asks the player if they want to play another round.
    Returns True to play again, False to quit.
    """
    try:
      play_again = input("\nWould you like to play? (y/n): ").strip().lower()
    except TypeError:
      print("wrong input try again")
      return ask_play()
    if play_again not in ['y', 'n']:
        print("Invalid input. Please enter 'y' to play again or 'n' to quit.")
        return ask_play()
    return play_again == 'y'

In [121]:
def main():
    # Initialize game components
    print("Welcome to Blackjack!")
    player = Player(name='Player1', balance=100)
    dealer = Dealer()
    deck = Deck()
    deck.shuffle()
    while True:  # Game loop for multiple rounds
        # Reset hands for a new round.
        time.sleep(5)
        clear_output()
        player.hand = []
        dealer.hand = []

        # Show player's balance and ask to play
        print(f"\nYour balance: {player.balance}$")
        if player.balance <= 0:
          print("You're out of money! Game over.")
          break

        elif not ask_play():
          break

        elif len(deck.all_cards) < 10:
          print("Deck is low in cards, shuffling...")
          deck = Deck()
          deck.shuffle()

        # Place a bet
        bet = player.add_bet()
        print(f"Bet placed: {bet}$")

        if not bet:
            continue

        # Deal initial cards and display hands
        deal_initial_cards(player, dealer, deck)
        display_initial_hands(player, dealer)

        # Check for Blackjack
        if check_blackjack(player, bet): continue

        # Player's turn
        elif handle_player_turn(player, deck): continue

        # Dealer's turn and determine winner
        handle_dealer_turn(dealer, deck)
        determine_winner(player, dealer, bet)

    print("Thank you for playing Blackjack!")


In [123]:
if __name__ == "__main__":
    main()